# Statistical Tests for Comparing Classification Algorithms
## Review of the seminal paper and implementations to uncover the best option for your data

Author: Tiago Toledo Jr.

Article from [towardsdatascience](https://towardsdatascience.com/statistical-tests-for-comparing-classification-algorithms-ac1804e79bb7).

> Note: In this notebook, I am studying the article mentioned above. Some changes may have been made to the code during its implementation.

# Initial Code Setup

In [1]:
# Importing the required libs
import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy.stats import norm, chi2
from scipy.stats import t as t_dist
from sklearn.datasets import load_wine
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, KFold

# Libs implementations
from mlxtend.evaluate import mcnemar
from mlxtend.evaluate import mcnemar_table
from mlxtend.evaluate import paired_ttest_5x2cv
from mlxtend.evaluate import proportion_difference
from mlxtend.evaluate import paired_ttest_kfold_cv
from mlxtend.evaluate import paired_ttest_resampled

# Getting the wine data from sklearn
X, y = load_wine(return_X_y=True)

# Instantiating the classification algorithms
rf = RandomForestClassifier(random_state=42)
knn = KNeighborsClassifier(n_neighbors=1)

# For holdout cases
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Two Proportions Test

In [2]:
# First we fit the classification algorithms
rf.fit(X_train, y_train)
knn.fit(X_train, y_train)

# Generate the predictions
rf_y = rf.predict(X_test)
knn_y = knn.predict(X_test)

# Calculate the accuracy
acc1 = accuracy_score(y_test, rf_y)
acc2 = accuracy_score(y_test, knn_y)

# Run the test
print('Proportions Z-Test')
z, p =proportion_difference(acc1, acc2, n_1=len(y_test))
print(f'z statistic: {z}, p-value: {p}\n')

Proportions Z-Test
z statistic: 3.2071349029490928, p-value: 0.9993296794413853



# Resampled Paired t-test

In [3]:
def paired_t_test(p):
    p_hat = np.mean(p)
    n = len(p)
    den = np.sqrt(sum([(diff - p_hat)**2 for diff in p]) / (n - 1))
    t = (p_hat * (n**(1/2))) / den

    p_value = t_dist.sf(t, n-1)*2

    return t, p_value

In [4]:
n_tests = 30

p_ = []
rng = np.random.RandomState(42)
for i in range(n_tests):
    randint = rng.randint(low=0, high=32767)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=randint)
    rf.fit(X_train, y_train)
    knn.fit(X_train, y_train)

    acc1 = accuracy_score(y_test, rf.predict(X_test))
    acc2 = accuracy_score(y_test, knn.predict(X_test))
    p_.append(acc1 - acc2)

print('Paired t-test Resampled')
t, p = paired_t_test(p_)
print(f't statistic: {t}, p-value: {p}\n')

Paired t-test Resampled
t statistic: 20.825868869252297, p-value: 5.467889135932688e-19



Below is the version already implemented in the Mlxtend library.

In [5]:
print('Paired t-test Resampled')
t, p = paired_ttest_resampled(estimator1=rf, estimator2=knn, X=X, y=y, random_seed=42, num_rounds=30, test_size=0.2)
print(f't statistic: {t}, p-value: {p}\n')

Paired t-test Resampled
t statistic: 20.825868869252297, p-value: 5.467889135932688e-19



# Cross-Validated Paired t-test

In [6]:
p_ = []

kf = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    rf.fit(X_train, y_train)
    knn.fit(X_train, y_train)

    acc1 = accuracy_score(y_test, rf.predict(X_test))
    acc2 = accuracy_score(y_test, knn.predict(X_test))
    p_.append(acc1 - acc2)

print('Cross Validated Paired t-tes')
t, p = paired_t_test(p_)
print(f't statistic: {t}, p-value: {p}\n')

Cross Validated Paired t-tes
t statistic: 9.468833293996424, p-value: 5.6252171240942e-06



Below is the version already implemented in the Mlxtend library.

In [7]:
t, p = paired_ttest_kfold_cv(estimator1=rf, estimator2=knn, X=X, y=y, random_seed=42, shuffle=True, cv=10)
print(f't statistic: {t}, p-value: {p}\n')

t statistic: 9.468833293996424, p-value: 5.6252171240942e-06



# McNemar's Test

In [26]:
def mcnemar_test(contingency_table, y_true, y_1, y_2):
    # b = sum(np.logical_and((knn_y != y_test),(rf_y == y_test)))
    # c = sum(np.logical_and((knn_y == y_test),(rf_y != y_test)))

    b = contingency_table[0][1]
    c = contingency_table[1][0]

    c_ = (np.abs(b - c) - 1)**2 / (b + c)

    p_value = chi2.sf(c_, 1)
    return c_, p_value

In [27]:
print('McNemar\'s test')

# Adjustment to generate the contingency table, as the original function was calculating the contingency table it was giving an error. So, the table is being generated automatically and the function that performs the calculation has been changed.
# For holdout cases
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

contingency_table = mcnemar_table(y_target=y_test, y_model1=rf_y, y_model2=knn_y)

chi2_, p = mcnemar_test(contingency_table, y_test, rf_y, knn_y)
print(f'chi2 statistic: {chi2_}, p-value: {p}\n')

McNemar's test
chi2 statistic: 6.125, p-value: 0.01332832878081758



Below is the version already implemented in the Mlxtend library.

In [28]:
print('McNemar\'s test')
table = mcnemar_table(y_target=y_test, y_model1=rf_y, y_model2=knn_y)
chi2_, p = mcnemar(ary=table, corrected=True)
print(f'chi2 statistic: {chi2_}, p-value: {p}\n')

McNemar's test
chi2 statistic: 6.125, p-value: 0.01332832878081758



# 5x2 Cross-Validation Test

In [31]:
def five_two_statistic(p1, p2):
    p1 = np.array(p1)
    p2 = np.array(p2)
    p_hat = (p1 + p2) / 2
    s = (p1 - p_hat)**2 + (p2 - p_hat)**2
    t = p1[0] / np.sqrt(1/5. * sum(s))

    p_value = t_dist.sf(t, 5)*2

    return t, p_value

In [33]:
p_1 = []
p_2 = []

rng = np.random.RandomState(42)
for i in range(5):
    randint = rng.randint(low=0, high=32767)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, random_state=randint)

    rf.fit(X_train, y_train)
    knn.fit(X_train, y_train)
    acc1 = accuracy_score(y_test, rf.predict(X_test))
    acc2 = accuracy_score(y_test, knn.predict(X_test))
    p_1.append(acc1 - acc2)

    rf.fit(X_test, y_test)
    knn.fit(X_test, y_test)
    acc1 = accuracy_score(y_train, rf.predict(X_train))
    acc2 = accuracy_score(y_train, knn.predict(X_train))
    p_2.append(acc1 - acc2)

# Running the test
print('5x2 CV Paired t-test')
t, p = five_two_statistic(p_1, p_2)
print(f't statistic: {t}, p-value: {p}\n')

5x2 CV Paired t-test
t statistic: 6.454972243679027, p-value: 0.0013279254349912806



Below is the version already implemented in the Mlxtend library.

In [34]:
print('5x2 CV Paired t-test')
t, p = paired_ttest_5x2cv(estimator1=rf, estimator2=knn, X=X, y=y, random_seed=42)
print(f't statistic: {t}, p-value: {p}\n')

5x2 CV Paired t-test
t statistic: 6.454972243679027, p-value: 0.0013279254349912806

